# SQL DataBase Formation

In [1]:
import sqlite3
import pandas as pd
import numpy as np
import os
import urllib2
import time
import datetime as dt
import requests
from bs4 import BeautifulSoup

In [2]:
## connect to database

path = "Data/mlb_data.db"
conn = sqlite3.connect(path)
c = conn.cursor()

In [3]:
def fangraphs_woba(df, season, fg_dict, fg_df):
    '''
    To add Fangraphs' woba value to each event.
    '''
    for i in range(0, len(df)):
        event = df.loc[[i], 'events'].values[0]
    
        if event in fg_dict.keys():
            df.loc[i, 'fg_woba_value'] = fg_df.loc[fg_df[fg_df['Season'] == season].index, fg_dict[event]].values[0]
        elif event == event:
            df.loc[i, 'fg_woba_value'] = 0 
    
    return df

## New method - from website

After browsing multiple scrapers on github, I chose to try to make my own. I decided to utilize the URL that Mr. Kessler used in his scraper (url below). I tried to make my own small loop scheme to import into a SQL database. I later realized it is similar to Mr. Kessler's. All credit for the link and method go to him and his scraper (namely, link, year/team loop idea, HTTPError catch and wait method).

reference: https://github.com/alanrkessler/savantscraper

### Fangraphs wOBA Values

First, though, I grab the Fangraphs wOBA values. As these change anually, and Statcast's data includes Standard wOBA values (http://tangotiger.com/index.php/site/comments/standard-woba), we may prefer to use FG's. values. 

I scrape the website for the values and form a dataframe with them, to map using a dictionary to the baseball savant events.

In [4]:
## get fangraphs woba values

link = 'https://www.fangraphs.com/guts.aspx?type=cn'

page = requests.get(link)
soup = BeautifulSoup(page.content, 'html.parser')

col_table = soup.find('thead')
cols = col_table.find_all('th', class_ = 'rgHeader')

df_columns = []

for stat in cols:
    df_columns.append(stat.text)
    
    
stat_table = soup.find(class_ = 'rgMasterTable')
stats1 = stat_table.find_all('tr', class_ = 'rgRow')
stats2 = stat_table.find_all('tr', class_ = 'rgAltRow')

stats = stats1 + stats2
counter = 0

for line in stats:
    temp = []
    
    for i in range(1, 15):
        temp.append(line.contents[i].text)
    
    temp = np.array(temp)
    
    if counter == 0:
        fg_df = pd.DataFrame(temp.reshape(-1, len(temp)), columns = df_columns)
        counter += 1
    else:
        fg_df = fg_df.append(pd.DataFrame(temp.reshape(-1, len(temp)), columns = df_columns))
    
fg_df = fg_df.sort_values('Season', ascending = False)
fg_df = fg_df.reset_index(drop = True)

### Baseball Savant Statcast

I scrape the Baseball Savant search for data from each team, for each season listed. I append the data to tables in my SQL Database. I also add two columns to the data - spray angle of the hit (estimated using hit location) and Fangraphs wOBA values.

In [5]:
## year_list & team_list & FG to savant woba list

year_list = [2015, 2016, 2017, 2018]

team_list = ['SF', 'LAD', 'ARI', 'COL', 'SD',
             'CHC', 'MIL', 'STL', 'CIN', 'PIT',
             'NYM', 'WSH', 'MIA', 'ATL', 'PHI',
             'OAK', 'HOU', 'LAA', 'TEX', 'SEA',
             'MIN', 'CWS', 'KC', 'DET', 'CLE',
             'NYY', 'BOS', 'TB', 'TOR', 'BAL']

savant_dict = {'walk': 'wBB',
              'single': 'w1B',
              'double': 'w2B',
              'triple': 'w3B',
              'home_run': 'wHR',
              'field_error': 'w1B',
              'hit_by_pitch': 'w1B',
              'catcher_interf': 'w1B'}


## loop for each team and year

for year in year_list:
    
    print(str(year) + ' Starting. Please wait' + 
          ' (5 to 15 minutes, depending on length of season and connection speed)...')

    counter = 1      # if first team of year, replace existing table

    for team in team_list:
        done = False     # if done, stop trying to access link (stays false if error)
                
        while not done:
            try:
                ## non-nan link
                link = 'https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR=' + \
                    '&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea=' + str(year) + \
                    '%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=' + \
                    '&game_date_gt=&game_date_lt=&team=' + team + \
                    '&position=&hfRO=&home_road=&hfFlag=&metric_1=&hfInn=&min_pitches=' + \
                    '0&min_results=0&group_by=name-event&sort_col=pitches&player_event_sort=' + \
                    'api_p_release_speed&sort_order=desc&min_abs=0&type=details&'
                
                ## nan-included link
#                 link = 'https://baseballsavant.mlb.com/statcast_search/csv?all=true&hfPT=&hfAB=&hfBBT=&hfPR' + \
#                     '=&hfZ=&stadium=&hfBBL=&hfNewZones=&hfGT=R%7C&hfC=&hfSea=' + str(year) + \
#                     '%7C&hfSit=&player_type=pitcher&hfOuts=&opponent=&pitcher_throws=&batter_stands=&hfSA=' + \
#                     '&game_date_gt=&game_date_lt=&hfInfield=&team=' + team + \
#                     '&position=&hfOutfield=&hfRO=&home_road=&hfFlag=&hfPull=&metric_1=&hfInn=&min_pitches=' + \
#                     '0&min_results=0&group_by=name&sort_col=pitches&player_event_sort=h_launch_speed&' + \
#                     'sort_order=desc&min_pas=0&type=details&'
                
                
                ## import data from link, a download csv link, and choose to replace table or append
            
                temp = pd.read_csv(link)
                exists = ('replace' if counter == 1 else 'append') 

                
                ## add columns - spray angle and fangraphs woba
                
                temp['spray_angle'] = \
    (np.arctan((temp['hc_x'] - 125.42)/(198.27 - temp['hc_y']))*180/np.pi*.75).apply(lambda x: round(x, 1))
    
                temp = temp.reset_index(drop = True)
                temp = fangraphs_woba(temp, year, savant_dict, fg_df) # user-defined function to add FG weighted woba
            
            
                ## import the data in the sql database and edit counter variables
            
                temp.to_sql("MLB_" + str(year), conn, if_exists=exists, index = False)  # import to SQL
                
                done = True      # if import and link work, done
                counter = counter + 1     # add to counter for each team completed
                
            except urllib2.HTTPError as e:     # catch an HTTP error if calling website too often
                print(e)
                print(str(year) + ' and ' + team + ' error...')
                time.sleep(5)     # wait a minute before trying again
        
    print(str(year) + ' Finished.')

2015 Starting. Please wait (5 to 15 minutes, depending on length of season and connection speed)...


IndexError: index 0 is out of bounds for axis 0 with size 0

## Old method 

I manually downloaded csv files and stored them in a directory, by year. From there, I would import each.

In [ ]:
# for year in range(2016, 2017):
    

#     ## current directory of data files

#     cd = "Data/savant/savant_" + str(year) + "/"     # change on your machine


#     ## create empty dataframe

#     data = pd.DataFrame()


#     ## loop through each sheet and append the previous one

#     for file_name in os.listdir(cd):
#         if 'DS' not in file_name:
#             import_data = pd.read_csv(cd + file_name).replace(
#                 'null', np.nan).convert_objects(convert_numeric = True)
#             data = data.append(import_data)


#     ## add spray angle

#     data['spray_angle'] = (np.arctan((data['hc_x'] - 125.42)/(198.27 - data['hc_y']))*180/np.pi*.75).apply(lambda x: round(x, 1))


#     # add dataframe to database

#     data.to_sql("MLB_" + str(year), conn, if_exists="replace", index = False)

## MLB ID key

To have a key to map names to numeric MLB player IDs.

source: http://crunchtimebaseball.com/baseball_map.html

In [ ]:
# ## current directory of data files

# cd = "http://crunchtimebaseball.com/master.csv"     # website of linked file


# ## create empty dataframe

# data = pd.read_csv(cd, encoding = 'latin-1').replace('null', np.nan).infer_objects()


# # add dataframe to database

# data.to_sql("ID_Key", conn, if_exists="replace", index = False)

## Database Checks

Check to see the tables listed to confirm their existance, and see the amount of data in each season table.

In [ ]:
c.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(c.fetchall())

In [ ]:
for year in [2015, 2016, 2017, 2018]:
    df = pd.read_sql("""SELECT game_date
        FROM MLB_{}
        ;""".format(year), conn)
    print year, len(df)

In [ ]:
## close access to database

c.close()